In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
def load_df():
    return pd.read_csv("../../data-final.csv", delimiter="\t")

def load_country_codes():
    return pd.read_csv("../../country_codes_2020.csv")

def load_population():
    return pd.read_csv("../../countries_by_population_2019.csv")

def load_happiness_level():
    return pd.read_csv("../../2019.csv")

In [3]:
df = load_df()
df

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,...,2016-03-03 02:01:01,768.0,1024.0,9.0,234.0,6,1,GB,51.5448,0.1991
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,...,2016-03-03 02:01:20,1360.0,768.0,12.0,179.0,11,1,MY,3.1698,101.706
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,...,2016-03-03 02:01:56,1366.0,768.0,3.0,186.0,7,1,GB,54.9119,-1.3833
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,...,2016-03-03 02:02:02,1920.0,1200.0,186.0,219.0,7,1,GB,51.75,-1.25
4,3.0,3.0,3.0,3.0,5.0,3.0,3.0,5.0,3.0,4.0,...,2016-03-03 02:02:57,1366.0,768.0,8.0,315.0,17,2,KE,1.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015336,4.0,2.0,4.0,3.0,4.0,3.0,3.0,3.0,3.0,3.0,...,2018-11-08 12:04:58,1920.0,1080.0,3.0,160.0,10,2,US,39.9883,-75.2208
1015337,4.0,3.0,4.0,3.0,3.0,3.0,4.0,4.0,3.0,3.0,...,2018-11-08 12:07:18,1920.0,1080.0,3.0,122.0,7,1,US,38.0,-97.0
1015338,4.0,2.0,4.0,3.0,5.0,1.0,4.0,2.0,4.0,4.0,...,2018-11-08 12:07:49,1920.0,1080.0,2.0,135.0,12,6,US,36.1473,-86.777
1015339,2.0,4.0,3.0,4.0,2.0,2.0,1.0,4.0,2.0,4.0,...,2018-11-08 12:08:34,1920.0,1080.0,6.0,212.0,8,1,US,34.1067,-117.8067


In [4]:
## Parameters

# Question columns
questions = df.columns[:50]

# Answering time columns
q_times = df.columns[50:100]

# Character traits
traits=['EXT','EST','AGR','CSN','OPN']

# Codebook.pkl attributes score points with respect to the personality trait (-1 if negative, 1 if positive)
codebook = pd.read_pickle('codebook.pkl')
codebook = codebook.map(lambda c : int(c))

# Display all the columns
pd.options.display.max_columns = None

In [5]:
# This function encompasses all the preprocessing that has been done during the step of the exploratory data analysis

def data_cleaning(df):
    
    # Keep only records where IPC = 1 and drop the NaNs
    df = df[df['IPC']== 1].dropna()
    
    # Drop the records where the localisation (country, latitude, longitude) is NONE
    df = df[df['country']!='NONE']
    df = df[df['lat_appx_lots_of_err']!='NONE']
    df = df[df['long_appx_lots_of_err']!='NONE']
    
    # set some variables as float
    df['endelapse'] = df['endelapse'].astype(float)
    df['lat_appx_lots_of_err'] = df['lat_appx_lots_of_err'].astype(float)
    df['long_appx_lots_of_err'] = df['long_appx_lots_of_err'].astype(float)

    # set categorical data
    # df['country'] = df['country'].astype(str).astype('category')

    # set datetime data
    df['dateload'] = pd.to_datetime(df['dateload'])
    
    # Discard the records where users didn't answer to more than 90% of the questions
    n_questions= 50
    thresold = n_questions*0.9
    df = df[(df[questions] > 0).sum(axis=1) > thresold]
    
    # We discard the records where there are some negative times (~400)
    df = df[(df[q_times]<0).sum(axis=1) == 0]

    # Discard all the records with time equal to 0 if the corresponding answer is not equal to 0 (~4000)
    df = df[((df[q_times]==0).sum(axis=1) == (df[questions]==0).sum(axis=1))]
    
    # We are going to keep countries which have submitted more than 100 answers    
    # Countries with more than 100 answers
    country_answers = df['country'].value_counts()
    country_answers = country_answers.where(country_answers>100).reset_index().rename(columns={"index": "country", "country": "count"})
    relevant_countries = country_answers.dropna()

    # Dropping the countries with less than 100 answers
    rel_countries_arr = relevant_countries['country'].array
    df = df[df['country'].isin(rel_countries_arr)]
    
    return df

In [6]:
def data_scoring_without_norm_per_country(df):
    
    # Computing the score
    scores=pd.DataFrame(0,index = df.index, columns= traits)
    for t in traits:
        trait_questions= [q for q in questions if t in q]
        for tq in trait_questions:
            scores[t] += df[tq]*codebook[tq]

    # Add them to the dataframe
    res = pd.concat([df,scores], axis = 1).drop(questions.append(q_times),axis = 1)
    res = res[['country','EXT','EST','AGR','CSN','OPN']]

    # Score per country
    res = res.groupby('country').mean()

    # Export the result to JSON file
    res.to_json(r'../Data/country_raw_mean_score.json',orient='index')

In [7]:
def data_scoring(df):
        
    # Computing the score
    scores=pd.DataFrame(0,index = df.index, columns= traits)
    for t in traits:
        trait_questions= [q for q in questions if t in q]
        for tq in trait_questions:
            scores[t] += df[tq]*codebook[tq]
            
    # Export the data to a pickle
    pd.concat([df,scores], axis = 1).drop(questions.append(q_times),axis = 1).to_pickle('scores.pkl')
    
    # Min Max normalization of the score
    scores = (scores-scores.min())/(scores.max()-scores.min())
    
    return scores

# Milestone 3 - Data wrangling

##### In this part, we are going to :
1. Prepare data for our questionary :
    - For each country, compute the mean score for each character trait.
    
    
2. Prepare data for our radar graph :
    - A. Character traits
        - Mean score of each character trait for each country
        - Mean score of each character trait globally
    - B. Response time
        - Mean response time for each character trait for each country
        - Mean response time for each character trait globally


3. Prepare data for our core visualization :
    - Dominant character traits for each country
    - The character trait associated to the longest response time for each country
    - Happiness level of each country
    
    
4. Prepare data for our barplot :
    - For each question, distribution of answers (#people who answered 1, #people who answered 2...) **globally/per country** (this will be interesting to see if there are some big disparities for some questions)
    

##### Optionally :
- Measure correlations between questions
- Clustering

### World population

To compute worldwide average, we need to do a weighted average of each country with its **population**.

In [8]:
pop_df = load_population().set_index("Rank")
pop_df = pop_df[['name','pop2019']]
pop_df['pop2019'] = pop_df['pop2019']*1000
pop_df

,name,pop2019
Rank,,
1,China,1.433784e+09
2,India,1.366418e+09
3,United States,3.290649e+08
4,Indonesia,2.706256e+08
5,Pakistan,2.165653e+08
...,...,...
228,Montserrat,4.989000e+03
229,Falkland Islands,3.377000e+03
230,Niue,1.615000e+03


Add the **country code** to it

In [9]:
country_codes = load_country_codes()
country_codes

,name,cca2,cca3,ccn3
0,Afghanistan,AF,AFG,4
1,Albania,AL,ALB,8
2,Algeria,DZ,DZA,12
3,American Samoa,AS,ASM,16
4,Andorra,AD,AND,20
...,...,...,...,...
227,Wallis and Futuna,WF,WLF,876
228,Western Sahara,EH,ESH,732
229,Yemen,YE,YEM,887
230,Zambia,ZM,ZMB,894


In [10]:
# Check which row has a NaN value
is_NaN = country_codes.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = country_codes[row_has_NaN]
rows_with_NaN

,name,cca2,cca3,ccn3
139,Namibia,NaN,NAM,516


In [11]:
# Replace the NaN value with the appropriate 2 letters country code 
country_codes.loc[country_codes['name']=='Namibia', 'cca2'] = 'NA'

In [12]:
# Add the country code to the population dataframe
pop_df = pd.merge(pop_df,
                  country_codes,
                  on='name',
                  how='left')
pop_df

,name,pop2019,cca2,cca3,ccn3
0,China,1.433784e+09,CN,CHN,156
1,India,1.366418e+09,IN,IND,356
2,United States,3.290649e+08,US,USA,840
3,Indonesia,2.706256e+08,ID,IDN,360
4,Pakistan,2.165653e+08,PK,PAK,586
...,...,...,...,...,...
227,Montserrat,4.989000e+03,MS,MSR,500
228,Falkland Islands,3.377000e+03,FK,FLK,238
229,Niue,1.615000e+03,NU,NIU,570
230,Tokelau,1.340000e+03,TK,TKL,772


## 1. Prepare data for our questionary

In [13]:
df = data_cleaning(df)
df.head()

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,EST1,EST2,EST3,EST4,EST5,EST6,EST7,EST8,EST9,EST10,AGR1,AGR2,AGR3,AGR4,AGR5,AGR6,AGR7,AGR8,AGR9,AGR10,CSN1,CSN2,CSN3,CSN4,CSN5,CSN6,CSN7,CSN8,CSN9,CSN10,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10,EXT1_E,EXT2_E,EXT3_E,EXT4_E,EXT5_E,EXT6_E,EXT7_E,EXT8_E,EXT9_E,EXT10_E,EST1_E,EST2_E,EST3_E,EST4_E,EST5_E,EST6_E,EST7_E,EST8_E,EST9_E,EST10_E,AGR1_E,AGR2_E,AGR3_E,AGR4_E,AGR5_E,AGR6_E,AGR7_E,AGR8_E,AGR9_E,AGR10_E,CSN1_E,CSN2_E,CSN3_E,CSN4_E,CSN5_E,CSN6_E,CSN7_E,CSN8_E,CSN9_E,CSN10_E,OPN1_E,OPN2_E,OPN3_E,OPN4_E,OPN5_E,OPN6_E,OPN7_E,OPN8_E,OPN9_E,OPN10_E,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,1.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,5.0,2.0,4.0,2.0,3.0,2.0,4.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0,4.0,4.0,2.0,4.0,4.0,5.0,1.0,4.0,1.0,4.0,1.0,5.0,3.0,4.0,5.0,9419.0,5491.0,3959.0,4821.0,5611.0,2756.0,2388.0,2113.0,5900.0,4110.0,6135.0,4150.0,5739.0,6364.0,3663.0,5070.0,5709.0,4285.0,2587.0,3997.0,4750.0,5475.0,11641.0,3115.0,3207.0,3260.0,10235.0,5897.0,1758.0,3081.0,6602.0,5457.0,1569.0,2129.0,3762.0,4420.0,9382.0,5286.0,4983.0,6339.0,3146.0,4067.0,2959.0,3411.0,2170.0,4920.0,4436.0,3116.0,2992.0,4354.0,2016-03-03 02:01:01,768.0,1024.0,9.0,234.0,6.0,1,GB,51.5448,0.1991
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,2.0,3.0,4.0,1.0,3.0,1.0,2.0,1.0,3.0,1.0,1.0,4.0,1.0,5.0,1.0,5.0,3.0,4.0,5.0,3.0,3.0,2.0,5.0,3.0,3.0,1.0,3.0,3.0,5.0,3.0,1.0,2.0,4.0,2.0,3.0,1.0,4.0,2.0,5.0,3.0,7235.0,3598.0,3315.0,2564.0,2976.0,3050.0,4787.0,3228.0,3465.0,3309.0,9036.0,2406.0,3484.0,3359.0,3061.0,2539.0,4226.0,2962.0,1799.0,1607.0,2158.0,2090.0,2143.0,2807.0,3422.0,5324.0,4494.0,3627.0,1850.0,1747.0,5163.0,5240.0,7208.0,2783.0,4103.0,3431.0,3347.0,2399.0,3360.0,5595.0,2624.0,4985.0,1684.0,3026.0,4742.0,3336.0,2718.0,3374.0,3096.0,3019.0,2016-03-03 02:01:20,1360.0,768.0,12.0,179.0,11.0,1,MY,3.1698,101.7060
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,1.0,4.0,1.0,4.0,2.0,4.0,1.0,4.0,4.0,3.0,4.0,2.0,2.0,2.0,3.0,3.0,4.0,2.0,4.0,2.0,5.0,1.0,2.0,1.0,4.0,2.0,5.0,3.0,4.0,4.0,4657.0,3549.0,2543.0,3335.0,5847.0,2540.0,4922.0,3142.0,14621.0,2191.0,5128.0,3675.0,3442.0,4546.0,8275.0,2185.0,2164.0,1175.0,3813.0,1593.0,1089.0,2203.0,3386.0,1464.0,2562.0,1493.0,3067.0,13719.0,3892.0,4100.0,4286.0,4775.0,2713.0,2813.0,4237.0,6308.0,2690.0,1516.0,2379.0,2983.0,1930.0,1470.0,1644.0,1683.0,2229.0,8114.0,2043.0,6295.0,1585.0,2529.0,2016-03-03 02:01:56,1366.0,768.0,3.0,186.0,7.0,1,GB,54.9119,-1.3833
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,3.0,3.0,3.0,2.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,4.0,3.0,4.0,2.0,4.0,2.0,4.0,3.0,4.0,2.0,4.0,4.0,4.0,1.0,2.0,2.0,3.0,1.0,4.0,4.0,2.0,5.0,2.0,3.0,1.0,4.0,4.0,3.0,3.0,3996.0,2896.0,5096.0,4240.0,5168.0,5456.0,4360.0,4496.0,5240.0,4000.0,3736.0,4616.0,3015.0,2711.0,3960.0,4064.0,4208.0,2936.0,7336.0,3896.0,6062.0,11952.0,1040.0,2264.0,3664.0,3049.0,4912.0,7545.0,4632.0,6896.0,2824.0,520.0,2368.0,3225.0,2848.0,6264.0,3760.0,10472.0,3192.0,7704.0,3456.0,6665.0,1977.0,3728.0,4128.0,3776.0,2984.0,4192.0,3480.0,3257.0,2016-03-03 02:02:02,1920.0,1200.0,186.0,219.0,7.0,1,GB,51.7500,-1.2500
5,3.0,3.0,4.0,2.0,4.0,2.0,2.0,3.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,3.0,1.0,4.0,2.0,3.0,2.0,3.0,4.0,4.0,3.0,2.0,4.0,1.0,3.0,2.0,4.0,3.0,4.0,3.0,5.0,1.0,5.0,1.0,3.0,1.0,5.0,4.0,5.0,2.0,4834.0,5064.0,1160.0,2664.0,6711.0,3344.0,2512.0,6264.0,6992.0,4592.0,2808.0,1776.0,3280.0,4520.0,2640.0,5408.0,3647.0,3183.0,1575.0,672.0,6375.0,4727.0,3775.0,1647.0,1233.0,8694.0,2904.0,2152.0,2856.0,2848.0,4288.0,4360.0,7328.0,3976.0,7895.0,2640.0,1760.0,5720.0,9032.0,3928.0,2104.0,5488.0,3656.0,4352.0,2681.0,3272.0,2640.0,1568.0,1640.0,3192.0,2016-03-03 02:03:12,1600.0,1000.0,4.0,196.0,3.0,1,SE,59.3333,18.0500


In [14]:
data_scoring_without_norm_per_country(df)

In [ ]:
# Translate the pickle file to JSON file

codebook_pickle = open("codebook.pkl","rb")
codebook = pickle.load(codebook_pickle)
codebook_df = pd.DataFrame.from_dict(codebook)
codebook_df.columns = ['score']
codebook_df.to_json(r'../Data/data_scoring.json',orient='index')

## 2. Prepare data for our radar graph

### Character trait

##### Mean score of each character trait for each country

In [15]:
df = data_cleaning(df)
df.head()

/home/kenkobz/Anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,EST1,EST2,EST3,EST4,EST5,EST6,EST7,EST8,EST9,EST10,AGR1,AGR2,AGR3,AGR4,AGR5,AGR6,AGR7,AGR8,AGR9,AGR10,CSN1,CSN2,CSN3,CSN4,CSN5,CSN6,CSN7,CSN8,CSN9,CSN10,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10,EXT1_E,EXT2_E,EXT3_E,EXT4_E,EXT5_E,EXT6_E,EXT7_E,EXT8_E,EXT9_E,EXT10_E,EST1_E,EST2_E,EST3_E,EST4_E,EST5_E,EST6_E,EST7_E,EST8_E,EST9_E,EST10_E,AGR1_E,AGR2_E,AGR3_E,AGR4_E,AGR5_E,AGR6_E,AGR7_E,AGR8_E,AGR9_E,AGR10_E,CSN1_E,CSN2_E,CSN3_E,CSN4_E,CSN5_E,CSN6_E,CSN7_E,CSN8_E,CSN9_E,CSN10_E,OPN1_E,OPN2_E,OPN3_E,OPN4_E,OPN5_E,OPN6_E,OPN7_E,OPN8_E,OPN9_E,OPN10_E,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,1.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,5.0,2.0,4.0,2.0,3.0,2.0,4.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0,4.0,4.0,2.0,4.0,4.0,5.0,1.0,4.0,1.0,4.0,1.0,5.0,3.0,4.0,5.0,9419.0,5491.0,3959.0,4821.0,5611.0,2756.0,2388.0,2113.0,5900.0,4110.0,6135.0,4150.0,5739.0,6364.0,3663.0,5070.0,5709.0,4285.0,2587.0,3997.0,4750.0,5475.0,11641.0,3115.0,3207.0,3260.0,10235.0,5897.0,1758.0,3081.0,6602.0,5457.0,1569.0,2129.0,3762.0,4420.0,9382.0,5286.0,4983.0,6339.0,3146.0,4067.0,2959.0,3411.0,2170.0,4920.0,4436.0,3116.0,2992.0,4354.0,2016-03-03 02:01:01,768.0,1024.0,9.0,234.0,6.0,1,GB,51.5448,0.1991
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,2.0,3.0,4.0,1.0,3.0,1.0,2.0,1.0,3.0,1.0,1.0,4.0,1.0,5.0,1.0,5.0,3.0,4.0,5.0,3.0,3.0,2.0,5.0,3.0,3.0,1.0,3.0,3.0,5.0,3.0,1.0,2.0,4.0,2.0,3.0,1.0,4.0,2.0,5.0,3.0,7235.0,3598.0,3315.0,2564.0,2976.0,3050.0,4787.0,3228.0,3465.0,3309.0,9036.0,2406.0,3484.0,3359.0,3061.0,2539.0,4226.0,2962.0,1799.0,1607.0,2158.0,2090.0,2143.0,2807.0,3422.0,5324.0,4494.0,3627.0,1850.0,1747.0,5163.0,5240.0,7208.0,2783.0,4103.0,3431.0,3347.0,2399.0,3360.0,5595.0,2624.0,4985.0,1684.0,3026.0,4742.0,3336.0,2718.0,3374.0,3096.0,3019.0,2016-03-03 02:01:20,1360.0,768.0,12.0,179.0,11.0,1,MY,3.1698,101.7060
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,1.0,4.0,1.0,4.0,2.0,4.0,1.0,4.0,4.0,3.0,4.0,2.0,2.0,2.0,3.0,3.0,4.0,2.0,4.0,2.0,5.0,1.0,2.0,1.0,4.0,2.0,5.0,3.0,4.0,4.0,4657.0,3549.0,2543.0,3335.0,5847.0,2540.0,4922.0,3142.0,14621.0,2191.0,5128.0,3675.0,3442.0,4546.0,8275.0,2185.0,2164.0,1175.0,3813.0,1593.0,1089.0,2203.0,3386.0,1464.0,2562.0,1493.0,3067.0,13719.0,3892.0,4100.0,4286.0,4775.0,2713.0,2813.0,4237.0,6308.0,2690.0,1516.0,2379.0,2983.0,1930.0,1470.0,1644.0,1683.0,2229.0,8114.0,2043.0,6295.0,1585.0,2529.0,2016-03-03 02:01:56,1366.0,768.0,3.0,186.0,7.0,1,GB,54.9119,-1.3833
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,3.0,3.0,3.0,2.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,4.0,3.0,4.0,2.0,4.0,2.0,4.0,3.0,4.0,2.0,4.0,4.0,4.0,1.0,2.0,2.0,3.0,1.0,4.0,4.0,2.0,5.0,2.0,3.0,1.0,4.0,4.0,3.0,3.0,3996.0,2896.0,5096.0,4240.0,5168.0,5456.0,4360.0,4496.0,5240.0,4000.0,3736.0,4616.0,3015.0,2711.0,3960.0,4064.0,4208.0,2936.0,7336.0,3896.0,6062.0,11952.0,1040.0,2264.0,3664.0,3049.0,4912.0,7545.0,4632.0,6896.0,2824.0,520.0,2368.0,3225.0,2848.0,6264.0,3760.0,10472.0,3192.0,7704.0,3456.0,6665.0,1977.0,3728.0,4128.0,3776.0,2984.0,4192.0,3480.0,3257.0,2016-03-03 02:02:02,1920.0,1200.0,186.0,219.0,7.0,1,GB,51.7500,-1.2500
5,3.0,3.0,4.0,2.0,4.0,2.0,2.0,3.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,3.0,1.0,4.0,2.0,3.0,2.0,3.0,4.0,4.0,3.0,2.0,4.0,1.0,3.0,2.0,4.0,3.0,4.0,3.0,5.0,1.0,5.0,1.0,3.0,1.0,5.0,4.0,5.0,2.0,4834.0,5064.0,1160.0,2664.0,6711.0,3344.0,2512.0,6264.0,6992.0,4592.0,2808.0,1776.0,3280.0,4520.0,2640.0,5408.0,3647.0,3183.0,1575.0,672.0,6375.0,4727.0,3775.0,1647.0,1233.0,8694.0,2904.0,2152.0,2856.0,2848.0,4288.0,4360.0,7328.0,3976.0,7895.0,2640.0,1760.0,5720.0,9032.0,3928.0,2104.0,5488.0,3656.0,4352.0,2681.0,3272.0,2640.0,1568.0,1640.0,3192.0,2016-03-03 02:03:12,1600.0,1000.0,4.0,196.0,3.0,1,SE,59.3333,18.0500


In [16]:
scores = data_scoring(df)
df_scores = pd.concat([df, scores],axis=1)
df_scores.head()

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,EST1,EST2,EST3,EST4,EST5,EST6,EST7,EST8,EST9,EST10,AGR1,AGR2,AGR3,AGR4,AGR5,AGR6,AGR7,AGR8,AGR9,AGR10,CSN1,CSN2,CSN3,CSN4,CSN5,CSN6,CSN7,CSN8,CSN9,CSN10,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10,EXT1_E,EXT2_E,EXT3_E,EXT4_E,EXT5_E,EXT6_E,EXT7_E,EXT8_E,EXT9_E,EXT10_E,EST1_E,EST2_E,EST3_E,EST4_E,EST5_E,EST6_E,EST7_E,EST8_E,EST9_E,EST10_E,AGR1_E,AGR2_E,AGR3_E,AGR4_E,AGR5_E,AGR6_E,AGR7_E,AGR8_E,AGR9_E,AGR10_E,CSN1_E,CSN2_E,CSN3_E,CSN4_E,CSN5_E,CSN6_E,CSN7_E,CSN8_E,CSN9_E,CSN10_E,OPN1_E,OPN2_E,OPN3_E,OPN4_E,OPN5_E,OPN6_E,OPN7_E,OPN8_E,OPN9_E,OPN10_E,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err,EXT,EST,AGR,CSN,OPN
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,1.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,5.0,2.0,4.0,2.0,3.0,2.0,4.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0,4.0,4.0,2.0,4.0,4.0,5.0,1.0,4.0,1.0,4.0,1.0,5.0,3.0,4.0,5.0,9419.0,5491.0,3959.0,4821.0,5611.0,2756.0,2388.0,2113.0,5900.0,4110.0,6135.0,4150.0,5739.0,6364.0,3663.0,5070.0,5709.0,4285.0,2587.0,3997.0,4750.0,5475.0,11641.0,3115.0,3207.0,3260.0,10235.0,5897.0,1758.0,3081.0,6602.0,5457.0,1569.0,2129.0,3762.0,4420.0,9382.0,5286.0,4983.0,6339.0,3146.0,4067.0,2959.0,3411.0,2170.0,4920.0,4436.0,3116.0,2992.0,4354.0,2016-03-03 02:01:01,768.0,1024.0,9.0,234.0,6.0,1,GB,51.5448,0.1991,0.863636,0.651163,0.720930,0.534884,0.833333
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,2.0,3.0,4.0,1.0,3.0,1.0,2.0,1.0,3.0,1.0,1.0,4.0,1.0,5.0,1.0,5.0,3.0,4.0,5.0,3.0,3.0,2.0,5.0,3.0,3.0,1.0,3.0,3.0,5.0,3.0,1.0,2.0,4.0,2.0,3.0,1.0,4.0,2.0,5.0,3.0,7235.0,3598.0,3315.0,2564.0,2976.0,3050.0,4787.0,3228.0,3465.0,3309.0,9036.0,2406.0,3484.0,3359.0,3061.0,2539.0,4226.0,2962.0,1799.0,1607.0,2158.0,2090.0,2143.0,2807.0,3422.0,5324.0,4494.0,3627.0,1850.0,1747.0,5163.0,5240.0,7208.0,2783.0,4103.0,3431.0,3347.0,2399.0,3360.0,5595.0,2624.0,4985.0,1684.0,3026.0,4742.0,3336.0,2718.0,3374.0,3096.0,3019.0,2016-03-03 02:01:20,1360.0,768.0,12.0,179.0,11.0,1,MY,3.1698,101.7060,0.272727,0.627907,0.837209,0.651163,0.595238
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,1.0,4.0,1.0,4.0,2.0,4.0,1.0,4.0,4.0,3.0,4.0,2.0,2.0,2.0,3.0,3.0,4.0,2.0,4.0,2.0,5.0,1.0,2.0,1.0,4.0,2.0,5.0,3.0,4.0,4.0,4657.0,3549.0,2543.0,3335.0,5847.0,2540.0,4922.0,3142.0,14621.0,2191.0,5128.0,3675.0,3442.0,4546.0,8275.0,2185.0,2164.0,1175.0,3813.0,1593.0,1089.0,2203.0,3386.0,1464.0,2562.0,1493.0,3067.0,13719.0,3892.0,4100.0,4286.0,4775.0,2713.0,2813.0,4237.0,6308.0,2690.0,1516.0,2379.0,2983.0,1930.0,1470.0,1644.0,1683.0,2229.0,8114.0,2043.0,6295.0,1585.0,2529.0,2016-03-03 02:01:56,1366.0,768.0,3.0,186.0,7.0,1,GB,54.9119,-1.3833,0.386364,0.604651,0.790698,0.581395,0.738095
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,3.0,3.0,3.0,2.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,4.0,3.0,4.0,2.0,4.0,2.0,4.0,3.0,4.0,2.0,4.0,4.0,4.0,1.0,2.0,2.0,3.0,1.0,4.0,4.0,2.0,5.0,2.0,3.0,1.0,4.0,4.0,3.0,3.0,3996.0,2896.0,5096.0,4240.0,5168.0,5456.0,4360.0,4496.0,5240.0,4000.0,3736.0,4616.0,3015.0,2711.0,3960.0,4064.0,4208.0,2936.0,7336.0,3896.0,6062.0,11952.0,1040.0,2264.0,3664.0,3049.0,4912.0,7545.0,4632.0,6896.0,2824.0,520.0,2368.0,3225.0,2848.0,6264.0,3760.0,10472.0,3192.0,7704.0,3456.0,6665.0,1977.0,3728.0,4128.0,3776.0,2984.0,4192.0,3480.0,3257.0,2016-03-03 02:02:02,1920.0,1200.0,186.0,219.0,7.0,1,GB,51.7500,-1.2500,0.409091,0.534884,0.697674,0.372093,0.690476
5,3.0,3.0,4.0,2.0,4.0,2.0,2.0,3.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,3.0,1.0,4.0,2.0,3.0,2.0,3.0,4.0,4.0,3.0,2.0,4.0,1.0,3.0,2.0,4.0,3.0,4.0,3.0,5.0,1.0,5.0,1.0,3.0,1.0,5.0,4.0,5.0,2.0,4834.0,5064.0,1160.0,2664.0,6711.0,3344.0,2512.0,6264.0,6992.0,4592.0,2808.0,1776.0,3280.0,4520.0,2640.0,5408.0,3647.0,3183.0,1575.0,672.0,6375.0,4727.0,3775.0,1647.0,1233.0,8694.0,2904.0,2152.0,2856.0,2848.0,4288.0,4360.0,7328.0,3976.0,7895.0,2640.0,1760.0,5720.0,9032.0,3928.0,2104.0,5488.0,3656.0,4352.0,2681.0,3272.0,2640.0,1568.0,1640.0,3192.0,2016-03-03 02:03:12,1600.0,1000.0,4.

In [17]:
country_score = df_scores.groupby('country').mean().iloc[:,-5:].dropna()
country_score

,EXT,EST,AGR,CSN,OPN
country,,,,,
AE,0.495253,0.469804,0.690683,0.575437,0.660845
AL,0.463911,0.479458,0.654313,0.556129,0.718883
AR,0.465901,0.456904,0.659159,0.525211,0.713289
AT,0.493501,0.511761,0.668409,0.542789,0.731411
AU,0.489265,0.497872,0.694576,0.569353,0.686811
...,...,...,...,...,...
US,0.486584,0.502260,0.704111,0.581290,0.699539
UY,0.471401,0.484343,0.668057,0.538842,0.702707
VE,0.476964,0.474604,0.638756,0.534154,0.708389


In [18]:
# Min Max normalization of the scores by country
n_country_score = (country_score-country_score.min())/(country_score.max()-country_score.min())
n_country_score.head()

,EXT,EST,AGR,CSN,OPN
country,,,,,
AE,0.692473,0.294502,0.646067,0.578721,0.540088
AL,0.443888,0.351554,0.382998,0.453011,0.877232
AR,0.459670,0.218275,0.418055,0.251710,0.844736
AT,0.678583,0.542437,0.484954,0.366154,0.950006
AU,0.644979,0.460362,0.674224,0.539108,0.690925


In [19]:
# Exporting the result to JSON format
n_country_score.to_json(r'../Data/country_score.json',orient='index')

##### Mean score of each character trait globally

In [20]:
# Check if there are countries that aren't in the population dataset
for c in df['country'].unique():
    if c not in pop_df['cca2'].unique():
        print("There is :", c)

In [21]:
# Add the population
country_score_pop = pd.merge(n_country_score,
                            pop_df[['name','cca2','pop2019']],
                            left_on='country',
                            right_on='cca2',
                            how='left').set_index('cca2')
country_score_pop.head()

,EXT,EST,AGR,CSN,OPN,name,pop2019
cca2,,,,,,,
AE,0.692473,0.294502,0.646067,0.578721,0.540088,United Arab Emirates,9770529.0
AL,0.443888,0.351554,0.382998,0.453011,0.877232,Albania,2880917.0
AR,0.459670,0.218275,0.418055,0.251710,0.844736,Argentina,44780677.0
AT,0.678583,0.542437,0.484954,0.366154,0.950006,Austria,8955102.0
AU,0.644979,0.460362,0.674224,0.539108,0.690925,Australia,25203198.0


In [22]:
# Weighted average per country
for t in traits:
    country_score_pop[t] = n_country_score[t]*country_score_pop['pop2019']
country_score_pop.head()

,EXT,EST,AGR,CSN,OPN,name,pop2019
cca2,,,,,,,
AE,6.765826e+06,2.877440e+06,6.312413e+06,5.654407e+06,5.276950e+06,United Arab Emirates,9770529.0
AL,1.278804e+06,1.012799e+06,1.103386e+06,1.305088e+06,2.527232e+06,Albania,2880917.0
AR,2.058435e+07,9.774481e+06,1.872079e+07,1.127176e+07,3.782784e+07,Argentina,44780677.0
AT,6.076779e+06,4.857576e+06,4.342815e+06,3.278946e+06,8.507402e+06,Austria,8955102.0
AU,1.625553e+07,1.160260e+07,1.699261e+07,1.358725e+07,1.741353e+07,Australia,25203198.0


In [23]:
# Compute the weighted average score
global_score = country_score_pop.copy()
# Keep only the relevant columns
global_score = global_score[traits+['pop2019']]
# Compute the sum
global_score = global_score.sum()
# Average
for t in traits:
    global_score[t] = global_score[t]/global_score['pop2019']
global_score = global_score[traits]
global_score

EXT    0.552951
EST    0.396178
AGR    0.500890
CSN    0.468555
OPN    0.583754
dtype: float64

In [24]:
# Exporting the result to JSON format
global_score.to_json(r'../Data/global_score.json',orient='index')

### Response time

##### Mean response time for each character trait for each country

In [25]:
# Compute the response time for each character trait
df_times = df.copy()
times = []
counter = 0
for t in traits:
    df_times[t+"_Time"] = df.iloc[:,50+counter*10 : 60+counter*10].sum(axis=1)
    counter += 1
df_times.head()

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,EST1,EST2,EST3,EST4,EST5,EST6,EST7,EST8,EST9,EST10,AGR1,AGR2,AGR3,AGR4,AGR5,AGR6,AGR7,AGR8,AGR9,AGR10,CSN1,CSN2,CSN3,CSN4,CSN5,CSN6,CSN7,CSN8,CSN9,CSN10,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10,EXT1_E,EXT2_E,EXT3_E,EXT4_E,EXT5_E,EXT6_E,EXT7_E,EXT8_E,EXT9_E,EXT10_E,EST1_E,EST2_E,EST3_E,EST4_E,EST5_E,EST6_E,EST7_E,EST8_E,EST9_E,EST10_E,AGR1_E,AGR2_E,AGR3_E,AGR4_E,AGR5_E,AGR6_E,AGR7_E,AGR8_E,AGR9_E,AGR10_E,CSN1_E,CSN2_E,CSN3_E,CSN4_E,CSN5_E,CSN6_E,CSN7_E,CSN8_E,CSN9_E,CSN10_E,OPN1_E,OPN2_E,OPN3_E,OPN4_E,OPN5_E,OPN6_E,OPN7_E,OPN8_E,OPN9_E,OPN10_E,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err,EXT_Time,EST_Time,AGR_Time,CSN_Time,OPN_Time
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,1.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,5.0,2.0,4.0,2.0,3.0,2.0,4.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0,4.0,4.0,2.0,4.0,4.0,5.0,1.0,4.0,1.0,4.0,1.0,5.0,3.0,4.0,5.0,9419.0,5491.0,3959.0,4821.0,5611.0,2756.0,2388.0,2113.0,5900.0,4110.0,6135.0,4150.0,5739.0,6364.0,3663.0,5070.0,5709.0,4285.0,2587.0,3997.0,4750.0,5475.0,11641.0,3115.0,3207.0,3260.0,10235.0,5897.0,1758.0,3081.0,6602.0,5457.0,1569.0,2129.0,3762.0,4420.0,9382.0,5286.0,4983.0,6339.0,3146.0,4067.0,2959.0,3411.0,2170.0,4920.0,4436.0,3116.0,2992.0,4354.0,2016-03-03 02:01:01,768.0,1024.0,9.0,234.0,6.0,1,GB,51.5448,0.1991,46568.0,47699.0,52419.0,49929.0,35571.0
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,2.0,3.0,4.0,1.0,3.0,1.0,2.0,1.0,3.0,1.0,1.0,4.0,1.0,5.0,1.0,5.0,3.0,4.0,5.0,3.0,3.0,2.0,5.0,3.0,3.0,1.0,3.0,3.0,5.0,3.0,1.0,2.0,4.0,2.0,3.0,1.0,4.0,2.0,5.0,3.0,7235.0,3598.0,3315.0,2564.0,2976.0,3050.0,4787.0,3228.0,3465.0,3309.0,9036.0,2406.0,3484.0,3359.0,3061.0,2539.0,4226.0,2962.0,1799.0,1607.0,2158.0,2090.0,2143.0,2807.0,3422.0,5324.0,4494.0,3627.0,1850.0,1747.0,5163.0,5240.0,7208.0,2783.0,4103.0,3431.0,3347.0,2399.0,3360.0,5595.0,2624.0,4985.0,1684.0,3026.0,4742.0,3336.0,2718.0,3374.0,3096.0,3019.0,2016-03-03 02:01:20,1360.0,768.0,12.0,179.0,11.0,1,MY,3.1698,101.7060,37527.0,34479.0,29662.0,42629.0,32604.0
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,1.0,4.0,1.0,4.0,2.0,4.0,1.0,4.0,4.0,3.0,4.0,2.0,2.0,2.0,3.0,3.0,4.0,2.0,4.0,2.0,5.0,1.0,2.0,1.0,4.0,2.0,5.0,3.0,4.0,4.0,4657.0,3549.0,2543.0,3335.0,5847.0,2540.0,4922.0,3142.0,14621.0,2191.0,5128.0,3675.0,3442.0,4546.0,8275.0,2185.0,2164.0,1175.0,3813.0,1593.0,1089.0,2203.0,3386.0,1464.0,2562.0,1493.0,3067.0,13719.0,3892.0,4100.0,4286.0,4775.0,2713.0,2813.0,4237.0,6308.0,2690.0,1516.0,2379.0,2983.0,1930.0,1470.0,1644.0,1683.0,2229.0,8114.0,2043.0,6295.0,1585.0,2529.0,2016-03-03 02:01:56,1366.0,768.0,3.0,186.0,7.0,1,GB,54.9119,-1.3833,47347.0,35996.0,36975.0,34700.0,29522.0
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,3.0,3.0,3.0,2.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,4.0,3.0,4.0,2.0,4.0,2.0,4.0,3.0,4.0,2.0,4.0,4.0,4.0,1.0,2.0,2.0,3.0,1.0,4.0,4.0,2.0,5.0,2.0,3.0,1.0,4.0,4.0,3.0,3.0,3996.0,2896.0,5096.0,4240.0,5168.0,5456.0,4360.0,4496.0,5240.0,4000.0,3736.0,4616.0,3015.0,2711.0,3960.0,4064.0,4208.0,2936.0,7336.0,3896.0,6062.0,11952.0,1040.0,2264.0,3664.0,3049.0,4912.0,7545.0,4632.0,6896.0,2824.0,520.0,2368.0,3225.0,2848.0,6264.0,3760.0,10472.0,3192.0,7704.0,3456.0,6665.0,1977.0,3728.0,4128.0,3776.0,2984.0,4192.0,3480.0,3257.0,2016-03-03 02:02:02,1920.0,1200.0,186.0,219.0,7.0,1,GB,51.7500,-1.2500,44948.0,40478.0,52016.0,43177.0,37643.0
5,3.0,3.0,4.0,2.0,4.0,2.0,2.0,3.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,3.0,1.0,4.0,2.0,3.0,2.0,3.0,4.0,4.0,3.0,2.0,4.0,1.0,3.0,2.0,4.0,3.0,4.0,3.0,5.0,1.0,5.0,1.0,3.0,1.0,5.0,4.0,5.0,2.0,4834.0,5064.0,1160.0,2664.0,6711.0,3344.0,2512.0,6264.0,6992.0,4592.0,2808.0,1776.0,3280.0,4520.0,2640.0,5408.0,3647.0,3183.0,1575.0,672.0,6375.0,4727.0,3775.0,1647.0,1233.0,8694.0,2904.0,2152.0,2856.0,2848.0,4288.0,4360.0,7328.0,3976.0,7895.0,2640.0,1760.0,5720.0,9032.0,3928.0,2104.0,5488.0,3656.0,4352.0,2681.0,3272.0,2640.0,1568.0,1640.0,3192.0,2016-03-03 02:03:12,1600.0,1000

In [26]:
country_time = df_times.groupby('country').mean().iloc[:,-5:].dropna()
country_time

,EXT_Time,EST_Time,AGR_Time,CSN_Time,OPN_Time
country,,,,,
AE,1.286582e+06,61175.667155,75491.737921,81581.478770,86013.057589
AL,6.904129e+04,57473.357349,61367.132565,83083.530259,59545.616715
AR,1.247762e+05,72207.330130,100029.659918,110575.508579,85478.889156
AT,7.089036e+04,58722.715101,82069.329528,74859.321225,57236.483134
AU,1.643743e+05,74697.174695,68979.235359,74474.065131,61342.037047
...,...,...,...,...,...
US,1.520331e+05,68397.669729,82681.506219,88463.794798,75539.305328
UY,2.122237e+05,62391.449848,65364.939210,76599.063830,90527.255319
VE,1.719538e+05,180721.432570,85282.033927,112889.375742,70366.413062


In [27]:
# Exporting the result to JSON format
country_time.to_json(r'../Data/country_time.json',orient='index')

##### Mean response time for each character trait globally

In [28]:
# Add the population
country_time_pop = pd.merge(country_time,
                            pop_df[['name','cca2','pop2019']],
                            left_on='country',
                            right_on='cca2',
                            how='left').set_index('cca2')
country_time_pop.head()

,EXT_Time,EST_Time,AGR_Time,CSN_Time,OPN_Time,name,pop2019
cca2,,,,,,,
AE,1.286582e+06,61175.667155,75491.737921,81581.478770,86013.057589,United Arab Emirates,9770529.0
AL,6.904129e+04,57473.357349,61367.132565,83083.530259,59545.616715,Albania,2880917.0
AR,1.247762e+05,72207.330130,100029.659918,110575.508579,85478.889156,Argentina,44780677.0
AT,7.089036e+04,58722.715101,82069.329528,74859.321225,57236.483134,Austria,8955102.0
AU,1.643743e+05,74697.174695,68979.235359,74474.065131,61342.037047,Australia,25203198.0


In [29]:
# Weighted average per country
time_traits = []
for t in traits :
    time_traits.append(t+'_Time')
for t in time_traits:
    country_time_pop[t] = country_time[t]*country_time_pop['pop2019']
country_time_pop.head()

,EXT_Time,EST_Time,AGR_Time,CSN_Time,OPN_Time,name,pop2019
cca2,,,,,,,
AE,1.257059e+13,5.977186e+11,7.375942e+11,7.970942e+11,8.403931e+11,United Arab Emirates,9770529.0
AL,1.989022e+11,1.655760e+11,1.767936e+11,2.393568e+11,1.715460e+11,Albania,2880917.0
AR,5.587565e+12,3.233493e+12,4.479396e+12,4.951646e+12,3.827803e+12,Argentina,44780677.0
AT,6.348304e+11,5.258679e+11,7.349392e+11,6.703729e+11,5.125585e+11,Austria,8955102.0
AU,4.142758e+12,1.882608e+12,1.738497e+12,1.876985e+12,1.546016e+12,Australia,25203198.0


In [30]:
# Compute the weighted average response time
global_time = country_time_pop.copy()
# Keep only the relevant columns
global_time = global_time[time_traits+['pop2019']]
# Compute the sum
global_time = global_time.sum()
# Average
for t in time_traits:
    global_time[t] = global_time[t]/global_time['pop2019']
global_time = global_time[time_traits]
global_time

EXT_Time    156518.522892
EST_Time     77949.260617
AGR_Time    119946.693915
CSN_Time     98244.589898
OPN_Time     86000.874246
dtype: float64

In [31]:
# Exporting the result to JSON format
global_time.to_json(r'../Data/global_time.json',orient='index')

## 3. Prepare data for our core visualization

##### Dominant character trait for each country

In [32]:
# Computing the distance between the global score and the country score
global_to_country_score_distance = n_country_score - global_score
global_to_country_score_distance

,EXT,EST,AGR,CSN,OPN
country,,,,,
AE,0.139522,-0.101676,0.145177,0.110166,-0.043666
AL,-0.109063,-0.044623,-0.117892,-0.015544,0.293478
AR,-0.093281,-0.177903,-0.082835,-0.216845,0.260982
AT,0.125632,0.146259,-0.015936,-0.102401,0.366252
AU,0.092027,0.064185,0.173334,0.070553,0.107171
...,...,...,...,...,...
US,0.070769,0.090116,0.242306,0.148272,0.181108
UY,-0.049660,-0.015760,-0.018480,-0.128095,0.199509
VE,-0.005534,-0.073310,-0.230417,-0.158619,0.232517


In [33]:
country_trait = global_to_country_score_distance.apply(lambda x: pd.Series(x.idxmax(), index=['dominant trait']), axis=1)
country_trait

,dominant trait
country,
AE,AGR
AL,OPN
AR,OPN
AT,OPN
AU,AGR
...,...
US,AGR
UY,OPN
VE,OPN


In [34]:
# Exporting the result to JSON format
country_trait.to_json(r'../Data/country_trait.json',orient='index')

##### Character trait associated to longest response time for each country

In [35]:
# Computing the distance between the global time and the country time
global_to_country_time_distance = country_time - global_time
global_to_country_time_distance

,EXT_Time,EST_Time,AGR_Time,CSN_Time,OPN_Time
country,,,,,
AE,1.130063e+06,-16773.593462,-44454.955994,-16663.111128,12.183343
AL,-8.747723e+04,-20475.903268,-58579.561350,-15161.059639,-26455.257531
AR,-3.174228e+04,-5741.930486,-19917.033997,12330.918681,-521.985090
AT,-8.562816e+04,-19226.545516,-37877.364387,-23385.268674,-28764.391111
AU,7.855770e+03,-3252.085922,-50967.458555,-23770.524767,-24658.837199
...,...,...,...,...,...
US,-4.485436e+03,-9551.590888,-37265.187696,-9780.795100,-10461.568917
UY,5.570518e+04,-15557.810769,-54581.754705,-21645.526068,4526.381073
VE,1.543532e+04,102772.171953,-34664.659988,14644.785844,-15634.461184


In [36]:
country_sensitive_trait = global_to_country_time_distance.apply(lambda x: pd.Series(x.idxmax()[:-5], index=['Trait associated to longest response time']), axis=1)
country_sensitive_trait

,Trait associated to longest response time
country,
AE,EXT
AL,CSN
AR,CSN
AT,EST
AU,EXT
...,...
US,EXT
UY,EXT
VE,EST


In [37]:
# Exporting the result to JSON format
country_sensitive_trait.to_json(r'../Data/country_sensitive_trait.json',orient='index')

##### Happiness level

In [38]:
# Load data
happiness_df = load_happiness_level()
# Keep useful columns
happiness_df = happiness_df[['Country or region','Overall rank','Score']]
# Rename columns
happiness_df = happiness_df.rename(columns={"Country or region": "name", "Overall rank": "rank", "Score": "score"})
# Add the country code to the happiness dataframe
happiness_df = pd.merge(happiness_df,
                        country_codes,
                        on='name',)
happiness_df = happiness_df[['cca2','score']].set_index('cca2')
happiness_df

,score
cca2,
FI,7.769
DK,7.600
NO,7.554
IS,7.494
NL,7.488
...,...
RW,3.334
TZ,3.231
AF,3.203


In [39]:
# Exporting the result to JSON format
happiness_df.to_json(r'../Data/country_happiness_level.json',orient='index')

## 4. Prepare data for our barplot

##### Distribution of the answers for each question, by country 

In [40]:
df.groupby('country')['EXT1'].value_counts().unstack().fillna(0)

EXT1,0.0,1.0,2.0,3.0,4.0,5.0
country,,,,,,
AE,2.0,475.0,338.0,668.0,360.0,206.0
AL,0.0,100.0,64.0,107.0,48.0,28.0
AR,1.0,865.0,488.0,952.0,439.0,169.0
AT,3.0,530.0,454.0,496.0,351.0,93.0
AU,31.0,8480.0,7196.0,9813.0,6818.0,1916.0
...,...,...,...,...,...,...
US,302.0,87889.0,70038.0,93401.0,65645.0,24593.0
UY,0.0,93.0,65.0,92.0,62.0,17.0
VE,0.0,344.0,185.0,453.0,145.0,52.0


In [41]:
# Generate the distribution of the answers for each question, by country, and export them
# for q in questions :
#    df.groupby('country')[q].value_counts().unstack().fillna(0).to_json(r'../Data/Country_Distributions/'+q+'.json',orient='index')
res = df.groupby('country')[questions[0]].value_counts().unstack().fillna(0)
res["question"] = questions[0]
res = res.reset_index()
res = pd.merge(res, country_codes, left_on='country', right_on = 'cca2', how = 'left')
res = res.iloc[:,:9]
res = res.set_index(['question','name','country'])
for q in range(1, len(questions)) :
    question_distrib = df.groupby('country')[questions[q]].value_counts().unstack().fillna(0)
    question_distrib["question"] = questions[q]
    question_distrib = question_distrib.reset_index()
    question_distrib = pd.merge(question_distrib, country_codes, left_on='country', right_on = 'cca2', how = 'left')
    question_distrib = question_distrib.iloc[:,:9]
    question_distrib = question_distrib.set_index(['question','name','country'])
    res = pd.concat([res, question_distrib])
res.to_json(r'../Data/country_distributions.json',orient='index')

In [42]:
# Generate the global distribution of the answers for each question, and export them
distribution_by_country = df.groupby('country')[questions[0]].value_counts().unstack().fillna(0)
global_distribution = distribution_by_country.sum()
global_distribution['question'] = questions[0]
global_distribution = global_distribution.to_frame().transpose().set_index(['question'])
for q in range(1, len(questions)) :
    distribution_by_country = df.groupby('country')[questions[q]].value_counts().unstack().fillna(0)
    temp = distribution_by_country.sum()
    temp['question'] = questions[q]
    temp = temp.to_frame().transpose().set_index(['question'])
    global_distribution = pd.concat([global_distribution, temp])
    
global_distribution.to_json(r'../Data/global_distributions.json',orient='index')

In [43]:
text = """EXT1	I am the life of the party.
EXT2	I don't talk a lot.
EXT3	I feel comfortable around people.
EXT4	I keep in the background.
EXT5	I start conversations.
EXT6	I have little to say.
EXT7	I talk to a lot of different people at parties.
EXT8	I don't like to draw attention to myself.
EXT9	I don't mind being the center of attention.
EXT10	I am quiet around strangers.
EST1	I get stressed out easily.
EST2	I am relaxed most of the time.
EST3	I worry about things.
EST4	I seldom feel blue.
EST5	I am easily disturbed.
EST6	I get upset easily.
EST7	I change my mood a lot.
EST8	I have frequent mood swings.
EST9	I get irritated easily.
EST10	I often feel blue.
AGR1	I feel little concern for others.
AGR2	I am interested in people.
AGR3	I insult people.
AGR4	I sympathize with others' feelings.
AGR5	I am not interested in other people's problems.
AGR6	I have a soft heart.
AGR7	I am not really interested in others.
AGR8	I take time out for others.
AGR9	I feel others' emotions.
AGR10	I make people feel at ease.
CSN1	I am always prepared.
CSN2	I leave my belongings around.
CSN3	I pay attention to details.
CSN4	I make a mess of things.
CSN5	I get chores done right away.
CSN6	I often forget to put things back in their proper place.
CSN7	I like order.
CSN8	I shirk my duties.
CSN9	I follow a schedule.
CSN10	I am exacting in my work.
OPN1	I have a rich vocabulary.
OPN2	I have difficulty understanding abstract ideas.
OPN3	I have a vivid imagination.
OPN4	I am not interested in abstract ideas.
OPN5	I have excellent ideas.
OPN6	I do not have a good imagination.
OPN7	I am quick to understand things.
OPN8	I use difficult words.
OPN9	I spend time reflecting on things.
OPN10	I am full of ideas.
"""

In [44]:
# Convert the corpus to a dictionary
tab = text.split("\n")
string = ""
dico = {}
for i in range(0, len(tab) - 1):
    temp = tab[i].split("\t")
    dico[temp[0]] = temp[1]

In [45]:
# Export it as a JSON file
import json
with open('../Data/questions_corpus.json', 'w') as fp:
    json.dump(dico, fp)